# Exercises for "Hands-on with Pydata: How to Build a Minimal Recommendation Engine"

# Systems check: imports and files

In [1]:
import numpy as np
import pandas as pd

# Pandas questions: Series and DataFrames
## 1. Adding a column in a DataFrame

In [2]:
# given the following DataFrame, add a new column to it
df = pd.DataFrame({'col1': [1,2,3,4]})
df['col2']=0
df

,col1,col2
0,1,0
1,2,0
2,3,0
3,4,0


## 2. Deleting a row in a DataFrame

In [3]:
# given the following DataFrame, delete row 'd' from it
df = pd.DataFrame({'col1': [1,2,3,4]}, index = ['a','b','c','d'])
df=df.drop('d')
df

,col1
a,1
b,2
c,3


## 3. Creating a DataFrame from a few Series

In [4]:
# given the following three Series, create a DataFrame such that it holds them as its columns
ser_1 = pd.Series(np.random.randn(6))
ser_2 = pd.Series(np.random.randn(6))
ser_3 = pd.Series(np.random.randn(6))
pd.DataFrame(data={'ser_1':ser_1,'ser_2':ser_2,'ser_3':ser_3})

,ser_1,ser_2,ser_3
0,0.299096,-0.039149,0.334821
1,0.085541,-0.636295,0.021511
2,-1.960442,0.368004,0.181735
3,-0.263605,1.339418,0.720917
4,-1.115136,-2.929530,-1.209504
5,-0.867969,-1.109559,0.534099


# Pandas questions: Indexing

## 1. Indexing into a specific column

In [5]:
# given the following DataFrame, try to index into the 'col_2' column
df = pd.DataFrame(data={'col_1': [0.12, 7, 45, 10], 'col_2': [0.9, 9, 34, 11]},
                  columns=['col_1', 'col_2', 'col_3'],
                  index=['obs1', 'obs2', 'obs3', 'obs4'])
df.loc[:,'col_2']

obs1     0.9
obs2     9.0
obs3    34.0
obs4    11.0
Name: col_2, dtype: float64

## 2. Label-based indexing

In [6]:
# using the same DataFrame, index into the row whose index is 'obs3'
df.loc['obs3']

col_1     45
col_2     34
col_3    NaN
Name: obs3, dtype: object

## 2. Position-based indexing

In [7]:
# using the same DataFrame, index into into its first row
df.iloc[0]

col_1    0.12
col_2     0.9
col_3     NaN
Name: obs1, dtype: object

# Mini-Challenge prep: data loading

## 1. How to load the `users` and `movies` portions of MovieLens

In [8]:
import pandas as pd

users = pd.read_table('data/ml-1m/users.dat',
                      sep='::', header=None,
                      names=['user_id', 'gender', 'age', 'occupation', 'zip'])

movies = pd.read_table('data/ml-1m/movies.dat',
                       sep='::', header=None,
                       names=['movie_id', 'title', 'genres'])

/Users/apple/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/Users/apple/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


## 2. How to load the training and testing subsets

In [9]:
# subset version (hosted notebook)
movielens_train = pd.read_csv('data/movielens_train.csv', index_col=0,encoding='latin-1')
movielens_test = pd.read_csv('data/movielens_test.csv', index_col=0,encoding='latin-1')

In [10]:
movielens_train.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
593263,3562,3798,4,967332344,F,25,6,32812,What Lies Beneath (2000),Thriller,False
235597,1051,3793,4,974958593,F,25,0,60513,X-Men (2000),Action|Sci-Fi,False
219003,3727,2366,3,966309522,M,35,7,74401,King Kong (1933),Action|Adventure|Horror,False
685090,4666,1094,3,963843918,M,35,1,53704,"Crying Game, The (1992)",Drama|Romance|War,False
312377,3261,1095,4,968251750,M,45,20,87505,Glengarry Glen Ross (1992),Drama,False


In [11]:
movielens_test.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
693323,4653,2648,4,975532459,M,35,12,95051,Frankenstein (1931),Horror,False
24177,2259,1270,4,974591524,F,56,16,70503,Back to the Future (1985),Comedy|Sci-Fi,False
202202,3032,1378,5,970343147,M,25,0,47303,Young Guns (1988),Action|Comedy|Western,False
262003,3029,2289,4,972846393,M,18,4,92037,"Player, The (1992)",Comedy|Drama,False
777848,4186,2403,3,1017931262,M,25,7,33308,First Blood (1982),Action,False


# Mini-Challenge prep: evaluation functions

These are the two functions that you will need to test your `estimate` method.

In [12]:
def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

In [13]:
def evaluate(estimate_f):
    """ RMSE-based predictive performance evaluation with pandas. """
    
    ids_to_estimate = zip(movielens_test.user_id, movielens_test.movie_id)
    estimated = np.array([estimate_f(u,i) for (u,i) in ids_to_estimate])
    real = movielens_test.rating.values
    return compute_rmse(estimated, real)//.001*.001, ((2668 - estimated.tolist().count(movielens_test['rating'].mean()))/2668)//.001*.1

Test a dummy solution!

In [14]:
def my_estimate_func(user_id, movie_id):
    return movielens_test['rating'].mean()

You can test for performance with the following line, which assumes that your function is called `my_estimate_func`:

In [18]:
print ('RMSE for my estimate function: {}'.format(evaluate(my_estimate_func)))

RMSE for my estimate function: (1.0880000000000001, 0.0)


# Reco systems questions: Minimal reco engine v1.0

In [16]:
movielens_train.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
593263,3562,3798,4,967332344,F,25,6,32812,What Lies Beneath (2000),Thriller,False
235597,1051,3793,4,974958593,F,25,0,60513,X-Men (2000),Action|Sci-Fi,False
219003,3727,2366,3,966309522,M,35,7,74401,King Kong (1933),Action|Adventure|Horror,False
685090,4666,1094,3,963843918,M,35,1,53704,"Crying Game, The (1992)",Drama|Romance|War,False
312377,3261,1095,4,968251750,M,45,20,87505,Glengarry Glen Ross (1992),Drama,False


## 1. Simple collaborative filtering using mean ratings

In [17]:
# write an 'estimate' function that computes the mean rating of a particular user
def collab_mean(user_id, movie_id):
    # first, index into all ratings of this movie
    ratings = movielens_test.loc[movielens_test['user_id']==user_id,'rating']
    # second, compute the mean of those ratings
    mean= np.mean(ratings)
    return mean

    
# try it out for a user_id, movie_id pair
collab_mean(4653, 2648)

4.0

# Mini-Challenge: first round
Implement an `estimate` function of your own using other similarity notions, eg.:

- collaborative filter based on age similarities
- collaborative filter based on zip code similarities
- collaborative filter based on occupation similarities
- content filter based on movie genre

In [289]:
def collab_age(user_id,movie_id):
    #get users age
    user_age = int(movielens_test.loc[movielens_test['user_id']==user_id,'age'].mean())
    #age condition
    age_con = movielens_test['age'] == user_age
    #user condition
    user_con = movielens_test['user_id'] != user_id
    #movie condition
    movie_con = movielens_test['movie_id'] == movie_id
    #other users with same age ratings
    other_user_ratings = movielens_test.loc[user_con & age_con & movie_con,'rating']
    if len(other_user_ratings)==0:
        return movielens_test['rating'].mean()
    else:
        return other_user_ratings.mean()
evaluate(collab_age)

(1.155, 35.800000000000004)

In [68]:
def collab_zip(user_id,movie_id):
    #get users age
    user_zip = movielens_test.loc[movielens_test['user_id']==user_id,'zip'].iloc[0]
    #age condition
    zip_con = movielens_test['zip'] == user_zip
    #user condition
    user_con = movielens_test['user_id'] != user_id
    #movie condition
    movie_con = movielens_test['movie_id'] == movie_id
    #other users with same age ratings
    other_user_ratings = movielens_test.loc[user_con & zip_con & movie_con,'rating']
    if len(other_user_ratings)==0:
        return movielens_test['rating'].mean()
    else:
        return other_user_ratings.mean()
evaluate(collab_zip)

(1.0880000000000001, 0.0)

In [291]:
def collab_occ(user_id,movie_id):
    #get users age
    user_occ = movielens_test.loc[movielens_test['user_id']==user_id,'occupation'].iloc[0]
    #age condition
    occ_con = movielens_test['occupation'] == user_occ
    #user condition
    user_con = movielens_test['user_id'] != user_id
    #movie condition
    movie_con = movielens_test['movie_id'] == movie_id
    #other users with same age ratings
    other_user_ratings = movielens_test.loc[user_con & occ_con & movie_con,'rating']
    if len(other_user_ratings)==0:
        return movielens_test['rating'].mean()
    else:
        return other_user_ratings.mean()
evaluate(collab_occ)

(1.1260000000000001, 13.9)

In [71]:
def content_genre(user_id,movie_id):
    #user cond
    user_con = user_con = movielens_test['user_id'] != user_id
    #movie condition
    movie_con = movielens_test['movie_id'] == movie_id
    #get movie genre
    if len(movielens_test.loc[user_con & movie_con,'genres'])>0:
        movie_genre = movielens_test.loc[user_con & movie_con,'genres'].iloc[0]
    else:
        movie_genre = 'none'
    #genre con
    if movie_genre == 'none':
        genre_con = 1==1
    else:
        genre_con = movielens_test['genres']== movie_genre
    #get ratings
    genre_ratings = movielens_test.loc[user_con & genre_con,'rating']
    if len(genre_ratings) == 0:
        return movielens_test['rating'].mean()
    else:
        return genre_ratings.mean()

evaluate(content_genre)

(1.1000000000000001, 99.9)

# Mini-Challenge: second round
Implement an `estimate` function of your own using other custom similarity notions, eg.:

- euclidean
- cosine

In [ ]:
def euclidean(s1, s2):
    """Take two pd.Series objects and return their euclidean 'similarity'."""
    diff = s1 - s2
    return 1 / (1 + np.sqrt(np.sum(diff ** 2)))